[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//gsg213/intro_to_PyTorch/blob/master/MLP.ipynb)

In [1]:
from torch.optim import SGD
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import torch.nn as nn
import torch
from collections import OrderedDict

Define the model

In [2]:
def get_model(inFeatures=4, hiddenDim=8, nbClasses=3):
	
  # build a sequential neural network based on input parameters
	
  mlp_model = nn.Sequential(OrderedDict([
		("hidden_layer_1", nn.Linear(inFeatures, hiddenDim)),
		("activation_1", nn.ReLU()),
		("output_layer", nn.Linear(hiddenDim, nbClasses))]))

	# return model
  return mlp_model

In [3]:
def next_batch(inputs, targets, batchSize):
	# loop over the dataset
	for i in range(0, inputs.shape[0], batchSize):
		# yield a tuple of the current batched data and labels
    #(return over iterations)
		yield (inputs[i:i + batchSize], targets[i:i + batchSize])

In [4]:
# define batch size, epochs, and learning rate
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Training using {}".format(DEVICE))

Training using cuda...


In [5]:
# generate a 3-class classification problem with 1000 data points,
# where each data point is a 4D feature vector
print("Preparing dataset")
X, y = make_blobs(n_samples=1000, n_features=4, centers=3, cluster_std=2.5, random_state=46)

# create training and testing splits, and convert them to PyTorch tensors
trainX, testX, trainY, testY = train_test_split(X, y,	test_size=0.15, random_state=46)
trainX = torch.from_numpy(trainX).float()
testX = torch.from_numpy(testX).float()
trainY = torch.from_numpy(trainY).float()
testY = torch.from_numpy(testY).float()
print("train size: {}".format(trainX.shape))
print("train size: {}".format(trainY.shape))

Preparing dataset
train size: torch.Size([850, 4])
train size: torch.Size([850])


In [6]:
# initialize model and display its architecture
#get_model(inFeatures=4, hiddenDim=8, nbClasses=3)
input_features = trainX.shape[1]
hidden_neurons = 12
output_classes = 3

mlp = get_model(inFeatures=input_features, hiddenDim= hidden_neurons, nbClasses=output_classes).to(DEVICE)
print(mlp)

# initialize optimizer and loss function
opt = SGD(mlp.parameters(), lr=LR)
lossFunc = nn.CrossEntropyLoss()

Sequential(
  (hidden_layer_1): Linear(in_features=4, out_features=12, bias=True)
  (activation_1): ReLU()
  (output_layer): Linear(in_features=12, out_features=3, bias=True)
)


Training


In [7]:

train_message = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
# loop through the epochs
for epoch in range(0, EPOCHS):
	# initialize tracker variables and set our model to trainable
	print("epoch: {}".format(epoch + 1))
	trainLoss = 0
	trainAcc = 0
	samples = 0
	mlp.train()
	# loop over the current batch of data
	for (batchX, batchY) in next_batch(trainX, trainY, BATCH_SIZE):
		# flash data to the current device, run it through our
		# model, and calculate loss
		(batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
		predictions = mlp(batchX)
		loss = lossFunc(predictions, batchY.long())
		# zero the gradients accumulated from the previous steps,
		# perform backpropagation, and update model parameters
		opt.zero_grad()
		loss.backward()
		opt.step()
		# update training loss, accuracy, and the number of samples
		# visited
		trainLoss += loss.item() * batchY.size(0)
		trainAcc += (predictions.max(1)[1] == batchY).sum().item()
		samples += batchY.size(0)
	# display model progress on the current training batch
	trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"
	print(trainTemplate.format(epoch + 1, (trainLoss / samples),
		(trainAcc / samples)))

epoch: 1
epoch: 1 train loss: 0.877 train accuracy: 0.622
epoch: 2
epoch: 2 train loss: 0.380 train accuracy: 0.940
epoch: 3
epoch: 3 train loss: 0.234 train accuracy: 0.972
epoch: 4
epoch: 4 train loss: 0.173 train accuracy: 0.976
epoch: 5
epoch: 5 train loss: 0.140 train accuracy: 0.978
epoch: 6
epoch: 6 train loss: 0.120 train accuracy: 0.979
epoch: 7
epoch: 7 train loss: 0.106 train accuracy: 0.979
epoch: 8
epoch: 8 train loss: 0.096 train accuracy: 0.979
epoch: 9
epoch: 9 train loss: 0.088 train accuracy: 0.979
epoch: 10
epoch: 10 train loss: 0.082 train accuracy: 0.980


Test


In [8]:
	# initialize tracker variables for testing, then set our model to
	# evaluation mode
	testLoss = 0
	testAcc = 0
	samples = 0
	mlp.eval()
	# initialize a no-gradient context, this doesn't aloow to modify the weights
	with torch.no_grad():
		# loop over the current batch of test data
		for (batchX, batchY) in next_batch(testX, testY, BATCH_SIZE):
			# flash the data to the current device
			(batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
			# run data through our model and calculate loss
			predictions = mlp(batchX)
			loss = lossFunc(predictions, batchY.long())
			# update test loss, accuracy, and the number of
			# samples visited
			testLoss += loss.item() * batchY.size(0)
			testAcc += (predictions.max(1)[1] == batchY).sum().item()
			samples += batchY.size(0)
		# display model progress on the current test batch
		testTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
		print(testTemplate.format(epoch + 1, (testLoss / samples),
			(testAcc / samples)))
		print("")

epoch: 10 test loss: 0.067 test accuracy: 0.993



Saving the model

In [15]:
torch.save(mlp.state_dict(),'tensor.pt')
#mlp.state_dict()

Loading the model


In [16]:
model = get_model(inFeatures=input_features, hiddenDim= hidden_neurons, nbClasses=output_classes).to(DEVICE)
model.load_state_dict(torch.load('tensor.pt'))
model.eval()

Sequential(
  (hidden_layer_1): Linear(in_features=4, out_features=12, bias=True)
  (activation_1): ReLU()
  (output_layer): Linear(in_features=12, out_features=3, bias=True)
)

In [17]:
	# initialize a no-gradient context, this doesn't aloow to modify the weights
	with torch.no_grad():
		# loop over the current batch of test data
		for (batchX, batchY) in next_batch(testX, testY, BATCH_SIZE):
			# flash the data to the current device
			(batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
			# we use the loaded model
			predictions = model(batchX)
			loss = lossFunc(predictions, batchY.long())
			# update test loss, accuracy, and the number of
			# samples visited
			testLoss += loss.item() * batchY.size(0)
			testAcc += (predictions.max(1)[1] == batchY).sum().item()
			samples += batchY.size(0)
		# display model progress on the current test batch
		testTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
		print(testTemplate.format(epoch + 1, (testLoss / samples),
			(testAcc / samples)))
		print("")

epoch: 10 test loss: 0.067 test accuracy: 0.993

